# Project 2 - Pandas for DS

## Part 1 - Olympics data

In [ ]:
import pandas as pd

In [1]:
def get_olympics_df():
    """
       The following code loads the olympics dataset (olympics.csv), which was derrived from the 
       Wikipedia entry (https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), 
       and does some basic data cleaning. 
       The columns are organized as 
           # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, 
           total # of medals. Use this dataset to answer the questions below.
    """
    df = pd.read_csv('./data/olympics.csv', index_col=0, skiprows=1)
    df.drop('Totals', axis=0, inplace=True) # :: footer line
    
    for col in df.columns:
        if col[:2]=='01':
            df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
        if col[:2]=='02':
            df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
        if col[:2]=='03':
            df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
        if col[:1]=='№':
            df.rename(columns={col:'#'+col[1:]}, inplace=True)

    names_ids = df.index.str.split('\s\(') # split the index by '('
    df.index = names_ids.str[0] # the [0] element is the country name (new index) 
    # Next 1 unused: This would return the country abbreviation first 3 characters
    #df['ID'] = names_ids.str[1].str[:3] 
    
    # the first country can be retrieved using df.iloc[0]:
    #   .iloc enables the access to values according to their position whereas
    #   .loc does the same but requires the value of the index at that position

    return df

In [2]:
df = get_olympics_df()
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12


In [3]:
def answer_one():
    # Q: country has won the most gold medals in summer games?
    # out: single string value
    #
    return df['Total'].argmax()

answer_one()

'United States'

In [4]:
def answer_two():
    # Q: country with the biggest difference between their summer and winter gold medal counts?
    # out: single string
    #
    import numpy as np
    df['Gold_diff'] = np.abs( df['Gold'] - df['Gold.1'] )
    out = df['Gold_diff'].argmax()
    return out 

answer_two()

'United States'

In [5]:
def answer_three():
    # Q: country that has the biggest difference between its summer and winter gold medal count
    #    relative to their total gold? Only include countries that have won at least 1 gold in 
    #    both summer and winter
    # out: single string value
    #
    import numpy as np
    
    df2 = df.where( (df['Gold'] > 0) & (df['Gold.1'] >0) ) # at least 1 winter and summer gold
    df2['Gold_diff2'] = ( df2['Gold'] - df2['Gold.1'] ) / ( df2['Gold'] + df2['Gold.1'] )
    out = df2['Gold_diff2'].argmax()
    return out

answer_three()

'Bulgaria'

In [6]:
def answer_four():
    #Q: create a Series called "Points" which is a weighted value where: 
    # gold medal (Gold.2)->3 points, silver medals (Silver.2)->2 points, and bronze medals (Bronze.2) ->1 point
    # out: a Series
    #
    Points = pd.Series( ( df['Gold.2']*3 + df['Silver.2']*2 + df['Bronze.2']) )
    return Points

answer_four()

Afghanistan                            2
Algeria                               27
Argentina                            130
Armenia                               16
Australasia                           22
Australia                            923
Austria                              569
Azerbaijan                            43
Bahamas                               24
Bahrain                                1
Barbados                               1
Belarus                              154
Belgium                              276
Bermuda                                1
Bohemia                                5
Botswana                               2
Brazil                               184
British West Indies                    2
Bulgaria                             411
Burundi                                3
Cameroon                              12
Canada                               846
Chile                                 24
China                               1120
Colombia        

## Part 2 - census data
The census data originates from the [United States Census Bureau](http://www.census.gov/popest/data/counties/totals/2015/CO-EST2015-alldata.html). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](http://www.census.gov/popest/data/counties/totals/2015/files/CO-EST2015-alldata.pdf) for a description of the variable names.

In [15]:
def get_census_df():
    df = pd.read_csv('./data/census.csv')
    return df

In [16]:
def answer_five():
    # Q: state with the most counties?
    # output: single string
    #
    census_df = get_census_df()
    
    # the SUMLEV variable has 2 values: 50 ('by county'), or 40 ('by state')
    by_cty_df = census_df[census_df['SUMLEV']==50 ][['STNAME', 'CTYNAME']]
    out = by_cty_df.groupby('STNAME').count()
    out_sorted = out.sort_values( by='CTYNAME', ascending=False)[:1]
    return out_sorted.index[0]
        
answer_five()

'Texas'

In [17]:
def answer_six():
    # Q: what are the three most populous states considering their 3 top most populous counties
    # output: a list of string values
    #
    census_df = get_census_df()
    
    def top(df, n=3, column='CENSUS2010POP'):
        return df.sort_values(by=column, ascending=False)[-n:]

    by_cty_df = census_df[census_df['SUMLEV']==50 ][['STNAME', 'CENSUS2010POP']]
    # take question literally & remove DC bc not a state?
    
    by_top3_cnties = by_cty_df.where(by_cty_df.STNAME != 'District of Columbia').groupby(['STNAME']).apply( top )
    out = by_top3_cnties.sum(level='STNAME').sort_values( by='CENSUS2010POP', ascending=False)[:3]

    return [out.index[0], out.index[1], out.index[2]]

answer_six()

['Delaware', 'Connecticut', 'New Jersey']

In [18]:
def answer_seven():
    # Q: county with the largest absolute change in population within the period 2010-2015?
    #    -> use columns POPESTIMATE2010 through POPESTIMATE2015.
    # out: single string value == the range of the row-wise series
    #
    census_df = get_census_df()

    by_cty_df = census_df[census_df['SUMLEV']==50 ][ census_df.columns[[6, 9, 10, 11, 12, 13, 14]] ]

    rng_cols = by_cty_df.columns[1:-1] #:: cols POPESTIMATE2010 through POPESTIMATE2015

    by_cty_df['pop_diff'] = by_cty_df[rng_cols].max(axis=1) - by_cty_df[rng_cols].min(axis=1)
    
    return by_cty_df.loc[ by_cty_df['pop_diff'].argmax()]['CTYNAME']

answer_seven()

'Harris County'

In [14]:
def answer_eight():
    # Q: query to retrieve:
    # the counties that belong to regions 1 OR 2, AND
    # whose name starts with 'Washington', AND
    # whose POPESTIMATE2015 > POPESTIMATE2014
    # output: a 5x2 DataFrame with ['STNAME', 'CTYNAME'] 
    # columns and the same index ID as the census_df (sorted ascending by index).
    #
    census_df = get_census_df()
    
    qry_df = census_df[ census_df.REGION.isin([1, 2]) & 
                        census_df.CTYNAME.str.startswith('Washington') & 
                        (census_df.POPESTIMATE2015 > census_df.POPESTIMATE2014) ][['STNAME', 'CTYNAME']]
    return qry_df

answer_eight()

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
